# Pristine Zr GB properties

In [ ]:
from pprint import pprint

import numpy as np

from utilities import (
    load_structures,
    get_all_interface_energies,
    get_all_wsep,
    get_interplanar_spacing_data,
    get_local_volume_change_data,
    get_coordination_change_data,
    get_structural_parameter_data,
    get_predicted_GB_widths,
    plot_GB_energies,
    plot_FS_energies,
    plot_works_of_separation,
    plot_vol_change,
    plot_coordination,
    plot_interplanar_spacing_data,
    get_binned_structures,
    get_atoms_num_electrons_by_voronoi,
)

In [ ]:
DFT_sims = load_structures(method="DFT", json_path="data/processed/DFT_sims.json")
EAM_sims = load_structures(method="EAM", json_path="data/processed/EAM_sims.json")

## Interface energetics

In [ ]:
# DFT predictions:
E_GB_DFT = get_all_interface_energies(DFT_sims, 'gb')
E_FS_DFT = get_all_interface_energies(DFT_sims, 'fs')

W_sep_DFT = get_all_wsep(DFT_sims)

# Get pristine W_GB and W_B:
W_GB_DFT = {k: v for k, v in W_sep_DFT.items() if len(k.split('-')) == 3 and '-gb' in k}
W_B_DFT = {k: v for k, v in W_sep_DFT.items() if len(k.split('-')) == 3 and '-b' in k}

# EAM E_GB predictions:
E_GB_EAM = get_all_interface_energies(EAM_sims, 'gb')

### Grain boundary energy

In [ ]:
plot_GB_energies(DFT_sims, E_GB_DFT)

### Surface energy

In [ ]:
plot_FS_energies(DFT_sims, E_FS_DFT)

### Work of separation - GB

In [ ]:
plot_works_of_separation(DFT_sims, W_GB_DFT)

### Work of separation - Bulk

In [ ]:
plot_works_of_separation(DFT_sims, W_B_DFT)

### Compare EAM- with DFT-predicted grain boundary energies

For EAM-DFT comparison, we compare grain boundary energies as predicted using the same methodology with both methods, i.e. from the minimum-energy γ-surface translation as predicted by the EAM or DFT methods, instead of the same γ-surface translation for both methods.

In [ ]:
E_GB_DFT['s7-tlA-gb'] = E_GB_DFT.pop('s7-tlAMin-gb')

In [ ]:
from utilities import write_manuscript_figure_E_GB_DFT_vs_EAM

fig = write_manuscript_figure_E_GB_DFT_vs_EAM(E_GB_EAM, E_GB_DFT, latex_labels=True)

## Geometric measures

In [ ]:
struct_params = get_structural_parameter_data()
lat_a_DFT, lat_c_DFT = struct_params['lattice_parameters']['dft'].values()
lat_a_EAM, lat_c_EAM = struct_params['lattice_parameters']['eam'].values()

vol_bulk_DFT = lat_a_DFT ** 2 * np.cos(np.deg2rad(30)) * lat_c_DFT / 2
vol_bulk_EAM = lat_a_EAM ** 2 * np.cos(np.deg2rad(30)) * lat_c_EAM / 2

### Interplanar spacing

In [ ]:
params = [
    {'structure_code': 's7-tlA-gb', 'step': 2, 'add_one': False, 'bulk_val': 1.05779},
    {'structure_code': 's7-tlB-gb', 'step': 2, 'add_one': False, 'bulk_val': 0.610715},
    {'structure_code': 's13-tlA-gb', 'step': 2, 'add_one': False, 'bulk_val': 0.7762077},
    {'structure_code': 's19-tlA-gb', 'step': 2, 'add_one': False, 'bulk_val': 0.642054},
    {'structure_code': 's31-tlA-gb', 'step': 2, 'add_one': False, 'bulk_val': 0.502652},
    {'structure_code': 's7-tw-gb', 'step': 1, 'add_one': True, 'bulk_val': lat_c_DFT/2, 'average_by': 7},
    {'structure_code': 's13-tw-gb', 'step': 1, 'add_one': True, 'bulk_val': lat_c_DFT/2, 'average_by': 13},
    {'structure_code': 's19-tw-gb', 'step': 1, 'add_one': True, 'bulk_val': lat_c_DFT/2, 'average_by': 19},
]

int_spacing_data_DFT = [get_interplanar_spacing_data(DFT_sims, **i) for i in params]
int_spacing_data_EAM = [get_interplanar_spacing_data(EAM_sims, **i) for i in params]

In [ ]:
plot_interplanar_spacing_data(int_spacing_data_DFT)

In [ ]:
plot_interplanar_spacing_data(int_spacing_data_EAM)

#### Manuscript figures

In [ ]:
from utilities import write_manuscript_figures_interplanar_spacing

figs = write_manuscript_figures_interplanar_spacing(DFT_sims, int_spacing_data_DFT, int_spacing_data_EAM)

### Change in local atomic volume

In [ ]:
structure_codes = (
    's7-tlA-gb',
    's7-tlB-gb',
    's13-tlA-gb',
    's19-tlA-gb',
    's31-tlA-gb',
    's7-tw-gb',
    's13-tw-gb',
    's19-tw-gb',
)
vol_change_DFT = [
    get_local_volume_change_data(DFT_sims, structure_code=i, vol_bulk=vol_bulk_DFT)
    for i in structure_codes
]
vol_change_EAM = [
    get_local_volume_change_data(EAM_sims, structure_code=i, vol_bulk=vol_bulk_EAM)
    for i in structure_codes
]

In [ ]:
plot_vol_change(vol_change_DFT)

In [ ]:
plot_vol_change(vol_change_EAM)

#### Predicted GB widths

In [ ]:
GB_widths_DFT, GB_widths_EAM = get_predicted_GB_widths(DFT_sims, vol_change_DFT, vol_change_EAM)

print('GB widths, lower and upper limits (DFT):')
pprint(GB_widths_DFT)

print('\nGB widths, lower and upper limits (EAM):')
pprint(GB_widths_DFT)

#### Manuscript figures

In [ ]:
from utilities import write_manuscript_figures_volume_change

figs = write_manuscript_figures_volume_change(DFT_sims, vol_change_DFT, vol_change_EAM)

### Coordination

In [ ]:
AREA_THRESHOLD = 1
structure_codes = (
    's7-tlA-gb',
    's7-tlB-gb',
    's13-tlA-gb',
    's19-tlA-gb',
    's31-tlA-gb',
    's7-tw-gb', 
    's13-tw-gb',
    's19-tw-gb',
)
coord_change_dat_DFT = [
    get_coordination_change_data(DFT_sims, structure_code=i, area_threshold=AREA_THRESHOLD)
    for i in structure_codes
]
coord_change_dat_EAM = [
    get_coordination_change_data(EAM_sims, structure_code=i, area_threshold=AREA_THRESHOLD)
    for i in structure_codes
]

In [ ]:
plot_coordination(coord_change_dat_DFT)

In [ ]:
plot_coordination(coord_change_dat_EAM)

#### Manuscript figures

In [ ]:
from utilities import write_manuscript_figures_coordination

figs = write_manuscript_figures_coordination(DFT_sims, coord_change_dat_DFT, coord_change_dat_EAM)

### Charge density analysis

In [ ]:
# Note:
#   - Binning the charge density in all structures below may take at least 30 minutes
#   - Binder may not have enough memory to bin all structures.

structure_codes = (
    's7-tlA-gb',
    's7-tlA-b',
    # 's7-tlB-gb',
    # 's7-tlB-b',
    # 's13-tlA-gb',
    # 's13-tlA-b',
    # 's19-tlA-gb',
    # 's19-tlA-b',
    # 's31-tlA-gb',
    # 's31-tlA-b', 
    # 's7-tw-gb',
    # 's7-tw-b',
    # 's13-tw-gb',
    # 's13-tw-b',
    # 's19-tw-gb',
    # 's19-tw-b',
)
binned_structs = get_binned_structures(structure_codes)

In [ ]:
# Find the value of `-min(Δnₑ)`
system_codes = set('-'.join(i.split('-')[:-1]) for i in binned_structs)
for i in system_codes:
    neg_min_dne = -np.min(
        np.array(list(get_atoms_num_electrons_by_voronoi(binned_structs[f'{i}-gb']).values()))
        - np.array(list(get_atoms_num_electrons_by_voronoi(binned_structs[f'{i}-b']).values()))
    )
    print(f'{i}: {neg_min_dne}')